# Installations and imports

In [1]:
!pip install pandas
!pip install icd10-cm
!pip install plotly
!pip install scikit-learn
!pip install xgboost
!pip install numpy
!pip install --upgrade nbformat

You should consider upgrading via the '/Users/amirgabay/Desktop/Work/Vi/Home assignment/Home Assignment/.venv3/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/amirgabay/Desktop/Work/Vi/Home assignment/Home Assignment/.venv3/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/amirgabay/Desktop/Work/Vi/Home assignment/Home Assignment/.venv3/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/amirgabay/Desktop/Work/Vi/Home assignment/Home Assignment/.venv3/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/amirgabay/Desktop/Work/Vi/Home assignment/Home Assignment/.venv3/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/amirgabay/Desktop/Work/Vi/Home assignment/Home Assignment/.venv3/bin/python3 -m pip install --upgrade pip' command.
You should consider upgradin

Attention: You may need to restart the kernel after installing all packages

In [2]:
# General imports

import pandas as pd
import icd10
import plotly.express as px

# Load the data

In [3]:
data_dir_path = "../data"

# load app_usage
app_usage_df = pd.read_csv(data_dir_path + "/app_usage.csv")
print(f"Loaded app_usage_df, shape: {app_usage_df.shape}")
# load claims
claims_df = pd.read_csv(data_dir_path + "/claims.csv")
print(f"Loaded claims_df, shape: {claims_df.shape}")
# load web_visits
web_visits_df = pd.read_csv(data_dir_path + "/web_visits.csv")
print(f"Loaded web_visits_df, shape: {web_visits_df.shape}")
# load churn_labels
churn_labels_df = pd.read_csv(data_dir_path + "/churn_labels.csv")
print(f"Loaded churn_labels_df, shape: {churn_labels_df.shape}")

Loaded app_usage_df, shape: (97789, 3)
Loaded claims_df, shape: (64576, 3)
Loaded web_visits_df, shape: (259340, 5)
Loaded churn_labels_df, shape: (10000, 4)


# EDA

In [4]:
# let's get the min and max dates of each df

# get the min and max timestamps of app_usage_df
app_usage_min_ts = app_usage_df['timestamp'].min()
app_usage_max_ts = app_usage_df['timestamp'].max()
print(f"app_usage_df timestamp range: {app_usage_min_ts} to {app_usage_max_ts}")

# get the min and max diagnosis date in claims_df
claims_min_ts = claims_df['diagnosis_date'].min()
claims_max_ts = claims_df['diagnosis_date'].max()
print(f"claims_df diagnosis date range: {claims_min_ts} to {claims_max_ts}")

# get the min and max timestamps in web_visits_df
web_visits_min_ts = web_visits_df['timestamp'].min()
web_visits_max_ts = web_visits_df['timestamp'].max()
print(f"web_visits_df timestamp range: {web_visits_min_ts} to {web_visits_max_ts}")

# get the min and max signup dates in churn_labels_df
churn_labels_min_ts = churn_labels_df['signup_date'].min()
churn_labels_max_ts = churn_labels_df['signup_date'].max()
print(f"churn_labels_df signup date range: {churn_labels_min_ts} to {churn_labels_max_ts}")

app_usage_df timestamp range: 2025-07-01 00:00:00 to 2025-07-14 23:59:51
claims_df diagnosis date range: 2025-07-01 to 2025-07-14
web_visits_df timestamp range: 2025-07-01 00:00:01 to 2025-07-14 23:59:49
churn_labels_df signup date range: 2024-01-01 to 2025-05-31


We can see that we have members that signed up from 2024-01-01 up until 2025-05-31. Other data sources show us their activity in the window between 2025-07-01 and 2025-07-15

In [5]:
# get icd code mapping using the icd10 library

icd_codes = claims_df['icd_code'].unique()
icd_code_mapping = {code: icd10.find(code).description for code in icd_codes}

icd_code_mapping

{'Z71.3': 'Dietary counseling and surveillance',
 'J00': 'Acute nasopharyngitis [common cold]',
 'M54.5': 'Low back pain',
 'I10': 'Essential (primary) hypertension',
 'E11.9': 'Type 2 diabetes mellitus without complications',
 'K21.9': 'Gastro-esophageal reflux disease without esophagitis',
 'R51': 'Headache',
 'A09': 'Infectious gastroenteritis and colitis, unspecified',
 'B34.9': 'Viral infection, unspecified',
 'H10.9': 'Unspecified conjunctivitis'}

In [6]:
# let's get the different possible claims and the number of claims made for each

claims_df["icd_description"] = claims_df["icd_code"].map(icd_code_mapping)
claims_df["icd_code_w_description"] = claims_df["icd_code"] + " - " + claims_df["icd_description"]

count_claims = claims_df.groupby("icd_code_w_description").size()
count_claims.sort_values(ascending=False, inplace=True)

fig = px.bar(x=count_claims.index, y=count_claims.values)
fig.update_layout(title="Number of Claims by ICD Code")
fig.update_layout(xaxis_title="ICD Code", yaxis_title="Number of Claims")
fig.show()

As states in the wellco_client_brief.txt we can see that there are three main icd-10 codes that received more claims than the others. In total there are 10 different icd-10 codes

In [7]:
# Show churn vs non-churned data

churn_counts = churn_labels_df["churn"].value_counts().reset_index()
churn_counts.columns = ["Churn Status", "Count"]

fig = px.pie(churn_counts, values="Count", names="Churn Status", title="Churned vs. Non-Churned Members")
# show labels and number
fig.update_traces(textinfo='percent+label+value')
# show column name in legend
fig.update_layout(legend_title_text='Churn Status')
fig.show()


We can see that we have inbalanced data. There are more members that did not chrn vs. members that churned

In [8]:
# show outreached vs. non-outreached members in pie chart

outreach_counts = churn_labels_df["outreach"].value_counts().reset_index()
outreach_counts.columns = ["Outreach Status", "Count"]

fig = px.pie(outreach_counts, values="Count", names="Outreach Status", title="Is outreached members deviation?")
# show labels and number
fig.update_traces(textinfo='percent+label+value')
# show column name in legend
fig.update_layout(legend_title_text='Outreach Status')
fig.show()

There are less outreached members (received treatment) than non-outreached members (did not receive treatment)

In [9]:
# let's get the number of visits to different urls per member

member_title_visits_df = web_visits_df.groupby(["member_id", "title"]).size().reset_index(name="title_num_visits_per_member").sort_values(by="title_num_visits_per_member", ascending=False)

# plot histogram of title_num_visits_per_member

fig = px.histogram(member_title_visits_df, x="title_num_visits_per_member", nbins=50, title="Number of Visits to a URL per Member")
fig.update_layout(xaxis_title="Number of Visits to a URL", yaxis_title="Count")
fig.show()


We can see that members usually visit a specific URL only a handful of times

In [10]:
# let's see how many times users interact with the app

member_web_visits_df = web_visits_df.groupby("member_id").size().reset_index(name="num_web_visits").sort_values(by="num_web_visits", ascending=False)

# plot histogram of num_web_visits

fig = px.histogram(member_web_visits_df, x="num_web_visits", nbins=50, title="Number of web visits per Member")
fig.update_layout(xaxis_title="Number of Web Visits", yaxis_title="Count")
fig.show()


In [11]:
# check the types of app usage

app_usage_df.groupby("event_type").size()

event_type
session    97789
dtype: int64

all app usage types are the "session" type

In [12]:
# How many members never interacted with the app

all_members_l = churn_labels_df["member_id"].unique()
app_interaction_members_l = app_usage_df["member_id"].unique()

members_never_interacted_l = set(all_members_l) - set(app_interaction_members_l)
num_members_never_interacted = len(members_never_interacted_l)

print(f"Number of members who never interacted with the app: {num_members_never_interacted} out of {len(all_members_l)} total members ({num_members_never_interacted/len(all_members_l)*100:.2f}%)")

Number of members who never interacted with the app: 2 out of 10000 total members (0.02%)


We can see that users tend to interact with the app for a few times. Some extreme cases show much higher or 0 times. Some users interacted only for one time with the app

In [13]:
# let's get the number of titles and the number of visits to them

num_visits_per_title_df = web_visits_df.groupby("title").size().reset_index(name="visit_count").sort_values(by="visit_count", ascending=False)

# plot the number of visits per title

fig = px.bar(num_visits_per_title_df, x="title", y="visit_count", title="Number of Web Visits per Title")
fig.update_layout(xaxis_title="Title", yaxis_title="Number of Web Visits")
fig.show()


# Preprocessing

## Feature Engineering

### Claims features

In [14]:
# Get for each member their number of claims in the observation window per icd_code
member_claims_per_icd_code_df = claims_df.groupby(["member_id", "icd_code"]).size().unstack(fill_value=0).reset_index()
# remove index name
member_claims_per_icd_code_df.columns.name = None
member_claims_per_icd_code_df

,member_id,A09,B34.9,E11.9,H10.9,I10,J00,K21.9,M54.5,R51,Z71.3
0,1,0,0,1,0,1,4,0,1,0,2
1,2,0,0,3,0,2,0,1,0,1,2
2,3,0,0,1,0,0,0,1,2,0,0
3,4,1,0,0,0,3,1,0,0,0,0
4,5,0,0,2,0,1,0,1,2,0,1
...,...,...,...,...,...,...,...,...,...,...,...
9975,9996,1,2,1,0,0,1,0,0,1,3
9976,9997,0,0,2,0,0,0,1,0,1,1
9977,9998,0,2,0,2,0,4,2,1,1,2
9978,9999,0,0,0,0,0,0,1,2,0,3


### App usage features

In [15]:
# let's get the app usage count in the observation window for each member

member_app_usage_count_df = app_usage_df.groupby("member_id").size().reset_index(name="app_usage_count").sort_values(by="app_usage_count", ascending=False)
member_app_usage_count_df

,member_id,app_usage_count
4989,4990,26
4467,4468,24
1418,1419,24
7833,7836,24
2097,2098,23
...,...,...
5011,5012,1
1633,1634,1
8735,8738,1
4892,4893,1


### Web visits features

In [16]:
# let's get the web visits count per member

member_web_visits_count_df = web_visits_df.groupby("member_id").size().reset_index(name="web_visits_count").sort_values(by="web_visits_count", ascending=False)
member_web_visits_count_df

,member_id,web_visits_count
112,113,140
4461,4475,132
2842,2850,124
9656,9682,122
9398,9424,122
...,...,...
7443,7463,1
7441,7461,1
3151,3159,1
3207,3215,1


In [17]:
# let's get the web visits per title per member
web_visits_per_title_per_member_df = web_visits_df.groupby(["member_id", "title"]).size().unstack(fill_value=0).reset_index()
# remove index name
web_visits_per_title_per_member_df.columns.name = None

web_visits_per_title_per_member_df


,member_id,Aerobic exercise,Budget planning,Cardio workouts,Cardiometabolic health,Cholesterol friendly foods,Diabetes management,Dog training,Electric vehicles,Exercise routines,...,Match highlights,Meditation guide,Mediterranean diet,New releases,Restorative sleep tips,Sleep hygiene,Strength training basics,Stress reduction,Top destinations,Weight management
0,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,1,1,0,0
2,3,0,2,3,1,1,0,4,2,3,...,4,3,3,4,3,1,0,0,3,0
3,4,0,2,0,1,0,1,1,3,0,...,0,1,0,0,1,1,1,1,1,0
4,5,0,0,0,0,0,1,1,0,1,...,1,0,0,1,1,1,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9970,9996,1,0,1,1,0,0,0,3,0,...,0,0,0,1,1,1,0,1,1,1
9971,9997,0,0,1,0,0,0,0,0,0,...,1,0,1,0,0,0,1,0,2,0
9972,9998,0,0,1,0,0,0,2,0,0,...,0,0,0,0,1,0,0,0,2,0
9973,9999,1,1,1,0,0,0,1,3,0,...,3,0,1,8,1,1,1,1,1,0


### Temporal features

We're predicting churn probability for right after the observation window that ends at 2025-07-15 00:00:00. Let's get temporal features of the number of days of each aspect until the end of the observation window.


In [18]:
# Define the observation window end timestamp
observation_window_end = pd.Timestamp("2025-07-15 00:00:00")

In [19]:
# Calculate the number of days since the last app usage
app_usage_df["timestamp"] = pd.to_datetime(app_usage_df["timestamp"])
latest_app_usage_df = app_usage_df.groupby("member_id")["timestamp"].max().reset_index(name="last_app_usage_timestamp")
latest_app_usage_df["days_since_last_app_usage"] = (observation_window_end - latest_app_usage_df["last_app_usage_timestamp"]).dt.total_seconds() / (24 * 3600)
latest_app_usage_df = latest_app_usage_df[["member_id", "days_since_last_app_usage"]]
latest_app_usage_df


,member_id,days_since_last_app_usage
0,1,0.354884
1,2,0.932350
2,3,2.762269
3,4,1.071007
4,5,1.078970
...,...,...
9993,9996,0.874178
9994,9997,1.103611
9995,9998,3.145278
9996,9999,1.090961


In [20]:
# Calculate the number of days since the last web visit
web_visits_df["timestamp"] = pd.to_datetime(web_visits_df["timestamp"])
latest_web_visit_df = web_visits_df.groupby("member_id")["timestamp"].max().reset_index(name="last_web_visit_timestamp")
latest_web_visit_df["days_since_last_web_visit"] = (observation_window_end - latest_web_visit_df["last_web_visit_timestamp"]).dt.total_seconds() / (24 * 3600)
latest_web_visit_df = latest_web_visit_df[["member_id", "days_since_last_web_visit"]]
latest_web_visit_df


,member_id,days_since_last_web_visit
0,1,0.973160
1,2,3.126377
2,3,0.117720
3,4,1.376076
4,5,1.049745
...,...,...
9970,9996,0.894016
9971,9997,3.803831
9972,9998,1.469606
9973,9999,0.210081


In [21]:
# Calculate the number of days since the last claim
claims_df["diagnosis_date"] = pd.to_datetime(claims_df["diagnosis_date"])
latest_claims_df = claims_df.groupby("member_id")["diagnosis_date"].max().reset_index(name="last_claim_date")
latest_claims_df["days_since_last_claim"] = (observation_window_end - latest_claims_df["last_claim_date"]).dt.total_seconds() / (24 * 3600)
latest_claims_df = latest_claims_df[["member_id", "days_since_last_claim"]]
latest_claims_df


,member_id,days_since_last_claim
0,1,1.0
1,2,2.0
2,3,1.0
3,4,2.0
4,5,1.0
...,...,...
9975,9996,3.0
9976,9997,1.0
9977,9998,1.0
9978,9999,2.0


In [22]:
# Calculate the number of days since the member has signed up

days_since_signup_df = churn_labels_df[["member_id", "signup_date"]].copy()
days_since_signup_df["signup_date"] = pd.to_datetime(days_since_signup_df["signup_date"])
days_since_signup_df["days_since_signup"] = (observation_window_end - days_since_signup_df["signup_date"]).dt.total_seconds() / (24 * 3600)
days_since_signup_df = days_since_signup_df[["member_id", "days_since_signup"]]
days_since_signup_df

,member_id,days_since_signup
0,1,459.0
1,2,126.0
2,3,291.0
3,4,455.0
4,5,490.0
...,...,...
9995,9996,316.0
9996,9997,524.0
9997,9998,317.0
9998,9999,541.0


## Create member level data

Let's create member level data to later train a model with

In [23]:
# get the labels data
data = churn_labels_df.copy()

# join the claims per icd code features
data = data.merge(member_claims_per_icd_code_df, on="member_id", how="left").fillna(0)

# join the app usage features
data = data.merge(member_app_usage_count_df, on="member_id", how="left").fillna(0)

# join the web visits features
data = data.merge(member_web_visits_count_df, on="member_id", how="left").fillna(0)

# # join the web visits per title count features
data = data.merge(web_visits_per_title_per_member_df, on="member_id", how="left").fillna(0)

# join the temporal features
data = data.merge(latest_app_usage_df, on="member_id", how="left").fillna(0)
data = data.merge(latest_web_visit_df, on="member_id", how="left").fillna(0)
data = data.merge(latest_claims_df, on="member_id", how="left").fillna(0)
data = data.merge(days_since_signup_df, on="member_id", how="left").fillna(0)

# remove the signup date feature - we don't need it as a model feature
data = data.drop(columns=["signup_date"])

data

,member_id,churn,outreach,A09,B34.9,E11.9,H10.9,I10,J00,K21.9,...,Restorative sleep tips,Sleep hygiene,Strength training basics,Stress reduction,Top destinations,Weight management,days_since_last_app_usage,days_since_last_web_visit,days_since_last_claim,days_since_signup
0,1,0,0,0.0,0.0,1.0,0.0,1.0,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.354884,0.973160,1.0,459.0
1,2,0,0,0.0,0.0,3.0,0.0,2.0,0.0,1.0,...,1.0,0.0,1.0,1.0,0.0,0.0,0.932350,3.126377,2.0,126.0
2,3,0,0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,3.0,1.0,0.0,0.0,3.0,0.0,2.762269,0.117720,1.0,291.0
3,4,1,0,1.0,0.0,0.0,0.0,3.0,1.0,0.0,...,1.0,1.0,1.0,1.0,1.0,0.0,1.071007,1.376076,2.0,455.0
4,5,0,1,0.0,0.0,2.0,0.0,1.0,0.0,1.0,...,1.0,1.0,1.0,0.0,1.0,1.0,1.078970,1.049745,1.0,490.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,0,0,1.0,2.0,1.0,0.0,0.0,1.0,0.0,...,1.0,1.0,0.0,1.0,1.0,1.0,0.874178,0.894016,3.0,316.0
9996,9997,0,0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,2.0,0.0,1.103611,3.803831,1.0,524.0
9997,9998,0,0,0.0,2.0,0.0,2.0,0.0,4.0,2.0,...,1.0,0.0,0.0,0.0,2.0,0.0,3.145278,1.469606,1.0,317.0
9998,9999,0,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,1.0,1.0,1.0,1.0,0.0,1.090961,0.210081,2.0,541.0


## Train/test split

We will use an uplift modeling approach. We will train two models - one for outreached members (treatment group) and one for non-outreached members (control group)

In [24]:
# let's split the data to train and test. We want it stratified on churn and outreach to have similar distributions in both sets
from sklearn.model_selection import train_test_split

id_cols = ["member_id"]
target = "churn"
treatment = "outreach"
cols = [col for col in data.columns if col not in id_cols + [target, treatment]]

train_df, test_df = train_test_split(
    data,
    test_size=0.25,
    stratify=data[["churn", "outreach"]],
    random_state=42
)

# check distribution of churn and outreach in train and test
print(f"Train churn distribution:\n{train_df['churn'].value_counts(normalize=True)}\n")
print(f"Test churn distribution:\n{test_df['churn'].value_counts(normalize=True)}\n")
print(f"Train outreach distribution:\n{train_df['outreach'].value_counts(normalize=True)}\n")
print(f"Test outreach distribution:\n{test_df['outreach'].value_counts(normalize=True)}\n")


Train churn distribution:
churn
0    0.797867
1    0.202133
Name: proportion, dtype: float64

Test churn distribution:
churn
0    0.798
1    0.202
Name: proportion, dtype: float64

Train outreach distribution:
outreach
0    0.6016
1    0.3984
Name: proportion, dtype: float64

Test outreach distribution:
outreach
0    0.6016
1    0.3984
Name: proportion, dtype: float64



Churn and outreach distribution are very similar as intended

# Model training and evaluation

## Train models

In [25]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from xgboost import XGBClassifier
import copy

potential_models = {
    "GradientBoosting": GradientBoostingClassifier(
       n_estimators=100,    # boost iterations
       max_depth=2,         # shallow trees generalize better
       random_state=42
    ),
    "RandomForest": RandomForestClassifier(
        n_estimators=200,         # more trees for stability
        max_depth=None,           # let them grow (can cap at 10–20 to avoid overfit)
        random_state=42
    ),
    "XGBoost": XGBClassifier(
        n_estimators=100,           # more boosting rounds
        max_depth=3,                # not too deep
        random_state=42,
        learning_rate=0.05
    )
}

def fit_control_and_treatment_models(train_df, model, cols, target, treatment):
    X_treatment = train_df.loc[train_df[treatment] == 1, cols]
    y_treatment = train_df.loc[train_df[treatment] == 1, target]
    X_control = train_df.loc[train_df[treatment] == 0, cols]
    y_control = train_df.loc[train_df[treatment] == 0, target]

    # create the two models based on the received model with its hyper parameters
    model_treatment = copy.deepcopy(model)
    model_control = copy.deepcopy(model)

    # fit the two models on the treatment and control groups respectively
    model_treatment.fit(X_treatment, y_treatment)
    model_control.fit(X_control, y_control)

    return model_treatment, model_control


trained_models = {}

for name, base_model in potential_models.items():
    print(f"Training models for {name}...")
    model_treatment, model_control = fit_control_and_treatment_models(train_df, base_model, cols, target, treatment)
    trained_models[name] = {
        "treatment": model_treatment,
        "control": model_control
    }


Training models for GradientBoosting...
Training models for RandomForest...
Training models for XGBoost...


## Evaluate the models

In [26]:
# Let's calculate the uplift and calculate the AUC score for each model

from sklearn.metrics import roc_auc_score

eval_rows = []

for name, models in trained_models.items():
    print(f"Evaluating model: {name}")
    model_treatment, model_control = models["treatment"], models["control"]
    X_test = test_df[cols]
    p1 = model_treatment.predict_proba(X_test)[:, 1]   # P(churn | outreach=1)
    p0 = model_control.predict_proba(X_test)[:, 1]   # P(churn | outreach=0)
    uplift = p0 - p1                        # expected churn reduction if treated. We subtract the treatment group probability from the control group probability. Since the treatment group is expected to have a lower churn rate and we want positive uplift.

    test_df[f"p1_{name}"] = p1
    test_df[f"p0_{name}"] = p0
    test_df[f"uplift_{name}"] = uplift

    # Let's see each model's ROC-AUC
    auc_t = roc_auc_score(test_df.loc[test_df[treatment]==1, target], p1[test_df[treatment]==1])
    auc_c = roc_auc_score(test_df.loc[test_df[treatment]==0, target], p0[test_df[treatment]==0])

    # print(f"Model: {name} - AUC Treatment: {auc_t:.4f}, AUC Control: {auc_c:.4f}")

    eval_rows.append({"model": name, "auc_t": auc_t, "auc_c": auc_c})


pd.DataFrame(eval_rows)


Evaluating model: GradientBoosting
Evaluating model: RandomForest
Evaluating model: XGBoost


,model,auc_t,auc_c
0,GradientBoosting,0.664645,0.664698
1,RandomForest,0.630966,0.642012
2,XGBoost,0.664471,0.664282


The models perform pretty similarly. Gradient boosting and XGBoost perform slightly better based on their ROC AUC scores. Let's see which model performs best when it comes to the possible uplift.

## Uplift evaluation

Since XGBoost has the best AUC scores we will use it as our model. We will now choose the n members to send them the outreach

In [27]:
import numpy as np
def get_qini_curve_values(df, uplift_col, treat_col="outreach", y_col="churn", bins=10):
    # Rank by predicted uplift (desc)
    d = df[[uplift_col, treat_col, y_col]].copy().sort_values(uplift_col, ascending=False)
    d["rank"] = np.arange(1, len(d)+1)
    # split into bins by rank percentiles
    d["bin"] = pd.qcut(d["rank"], q=bins, labels=False)  # 0..bins-1

    out = []
    cum_inc = 0.0
    cum_n = 0
    for b in range(bins):
        group = d[d["bin"] <= b]  # cumulative top-x%
        treated = group[group[treat_col]==1]
        control = group[group[treat_col]==0]

        # avoid div by zero
        if len(treated)==0 or len(control)==0:
            out.append({"bin": b+1, "pct": (b+1)/bins, "cum_incremental": np.nan})
            continue

        # churn rates
        cr_t = treated[y_col].mean()
        cr_c = control[y_col].mean()

        # incremental churns prevented = (cr_c - cr_t) * group_size
        inc = (cr_c - cr_t) * len(group)
        out.append({"bin": b+1, "pct": (b+1)/bins, "n": len(group), "cum_incremental": inc})

    qini = pd.DataFrame(out)
    # baseline (random) is a straight line from 0 to max(cum_incremental)
    return qini



In [28]:
# plot the qini curve
import plotly.express as px

import numpy as np
import plotly.express as px
import plotly.graph_objects as go

def plot_qini_curve(qini_df, title="Qini Curve", x_col="pct", y_col="cum_incremental", add_baseline=True):
    # Ensure sorted by x
    qini_df = qini_df.sort_values(x_col).reset_index(drop=True)

    # Main curve
    fig = px.line(
        qini_df,
        x=x_col,
        y=y_col,
        markers=True,
        title=title,
        labels={x_col: "Fraction targeted" if x_col == "pct" else "Number targeted",
                y_col: "Cumulative incremental gain",
                "n": "Number of samples"},
        hover_data={"n": True}
    )

    # Random baseline: straight line from (0,0) to (x_last, y_last)
    if add_baseline:
        x = qini_df[x_col].values
        y_end = qini_df[y_col].iloc[-1]
        y_rand = (x / x.max()) * y_end if x.max() != 0 else np.zeros_like(x)
        fig.add_trace(go.Scatter(x=x, y=y_rand, mode="lines", name="Random baseline", line=dict(dash="dash")))

    # add vertical line along the x-axis for max uplift point
    fig.add_vline(x=qini_df[x_col].iloc[qini_df[y_col].idxmax()], line=dict(dash="dash", color="green"))

    fig.update_layout(showlegend=True)
    return fig


for name, model in trained_models.items():
    uplift_col = f"uplift_{name}"
    qini_values = get_qini_curve_values(test_df, uplift_col)
    fig = plot_qini_curve(qini_values, title=f"Qini Curve for {name}")
    fig.show()

The qini curve shows us that with XGBoost we managed to get to the highest uplift (saved churns) by only outreaching to the top 50% members in our test set

# Create a final outreach ranked list

Our test set simulates a set of members that are possible to be outreached in production. The train set is the samples our models are trained on and we assume that these are members from the past that we can learn from, but it is not possible to outreach them in production.

So our ranked list is built on the test set, and only for users in the test set that haven't received an outreach yet (because we assume that we can't outreach to a member we already outreached to).

In [29]:
# Let's get the uplift of all members in the test set

to_save_df = test_df.copy()

# We rank based on the XGBoost related columns
to_save_df = to_save_df.sort_values("uplift_XGBoost", ascending=False, inplace=False)
# let's take only the top 50% - as shown in the qini curve
to_save_df = to_save_df.reset_index(drop=True)
to_save_df = to_save_df.iloc[:int(len(to_save_df)*0.5), :]
# remove outreached members - we can't outreach members that were already outreached
to_save_df = to_save_df[to_save_df["outreach"] == 0].reset_index(drop=True)
# let's take only the necessary columns
to_save_df = to_save_df[["member_id", "uplift_XGBoost"]]
to_save_df["rank"] = to_save_df["uplift_XGBoost"].rank(ascending=False)
# rename uplift column
to_save_df = to_save_df.rename(columns={"uplift_XGBoost": "prioritization_score"})
# change rank column to int
to_save_df["rank"] = to_save_df["rank"].astype(int)
to_save_df

,member_id,prioritization_score,rank
0,121,0.290031,1
1,3195,0.265913,2
2,8746,0.253960,3
3,1496,0.247868,4
4,8005,0.230059,5
...,...,...,...
760,5604,0.032306,761
761,9654,0.032162,762
762,5373,0.032079,763
763,5202,0.031917,764


In [30]:
# save the final dataframe
print(f"Saving prioritized outreach members to prioritized_outreach_members.csv...")
to_save_df.to_csv("prioritized_outreach_members.csv", index=False)
print(f"Finished saving prioritized outreach members.")

Saving prioritized outreach members to prioritized_outreach_members.csv...
Finished saving prioritized outreach members.
